# ОПИСАНИЕ ЗАДАЧИ

Данные:
- чековые данные (transactions.parquet, для чтения через pandas дополнительно нужно установить библиотеку pyarrow)
- справочник товаров (materials.csv)
- справочник магазинов (plants.csv)
- справочник клиентов (clients.csv)
Более подробное описание данных дано в файле Data Description.

Цель: 
1) проанализировать данные и определить оптимальную методологию определения отточных клиентов
2) разработать модель вероятности оттока клиентов по выбранной вами методологии
3) дать интерпретацию разработанной модели, ответить на вопросы: какие признаки наиболее влияют на отток клиентов

# ЧАСТЬ 3. ФОРМИРОВАНИЕ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ

### Загрузка библиотек

In [8]:
import pandas as pd
from pathlib import Path
import numpy as np
import pickle
import gc
from statistics import mode
from tqdm.notebook import tqdm


pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings("ignore")

# 1. Загрузка данных

In [10]:
workdir = str(Path().absolute())

In [11]:
full = pd.read_pickle(workdir+'/data/full.pkl')

Посмотрим на данные.

In [12]:
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32094659 entries, 0 to 32094658
Data columns (total 18 columns):
chq_date            datetime64[ns]
chq_position        int32
sales_count         float16
sales_sum           float32
is_promo            bool
client_id           int32
material            int32
plant               int16
chq_id              int32
price               float32
hier_level_1        int8
hier_level_2        int8
hier_level_3        int16
hier_level_4        int16
vendor              int16
is_private_label    bool
is_alco             bool
plant_type          int8
dtypes: bool(3), datetime64[ns](1), float16(1), float32(2), int16(4), int32(4), int8(3)
memory usage: 1.7 GB


In [13]:
print(80*'*')
print('full:')
display(full.head())

********************************************************************************
full:


,chq_date,chq_position,sales_count,sales_sum,is_promo,client_id,material,plant,chq_id,price,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco,plant_type
0,2016-11-01,4,2.0,146.979996,False,56437,14213,179,2390608,73.489998,0,32,130,936,2736,False,False,0
1,2016-11-01,3,1.0,249.990005,True,56437,7629,179,2390608,249.990005,0,10,305,1103,2278,False,False,0
2,2016-11-01,12,1.0,47.990002,True,56437,7264,179,2390608,47.990002,0,13,81,3,2715,False,True,0
3,2016-11-01,13,1.0,47.990002,True,56437,7264,179,2390608,47.990002,0,13,81,3,2715,False,True,0
4,2016-11-01,11,2.0,53.980000,True,56437,35784,179,2390608,26.990000,0,32,383,1775,1221,False,False,0


# 2. Выбор стратегии

### 2.1. Целевая переменная

Целевая переменная представляет собой двоичную классифицию клиента по состоянию "активность" / "отток":
* 0 - активный клиент
* 1 - отточный клиент

Стратегия определения отточного клиента в известных данных:
* _Группа 1: Клиенты, переставшие покупать._ Клиент, не сделавший покупок в течение 31 дней, считается отточным в течение этого обозреваемого периода. Период в 31 дней позволит учитывать активность клиентов, делающих покупки раз в месяц, и клиентов, находящихся в месячном отпуске. 
* _Группа 2: Клиенты, ставшие покупать меньше._ Например, это могут быть клиенты, которых больше не устраивают условия (ассортимент товаров, ценовая политика, обслуживание), но они продолжают совершать небольшие покупки из-за удобства расположения магазина. Это менее различимая группа. Но можно попробовать находить хотя бы часть таких клиентов. Если клиент, в течение 31 дней, делал покупки на общую ежедневную сумму ниже пороговой суммы, то он считается отточным в течение этого обозреваемого периода. Поиск оптимальной пороговой суммы представляется трудоемким и длительным процессом, выходящим за рамки данной задачи. Поэтому сделаем его очень низким: пороговая сумма будет определяться для клиента как 25% от средней суммы ежедневных покупок за весь известный период.

### 2.2. Возможные ошибки классификации

* Клиенты, делающие покупки раз в месяц, сделали две соседние покупки с интервалом более 31 дней. Данную ошибку можно избежать, анализируя активность клиентов за прошедшие 2 месяца. Однако, в рамках текущей задачи эта возможная ошибка классификации не учитывается в связи с нехваткой аппаратных ресурсов.
* Для клиентов, активных в первый месяц собранных данных, условно считаем, что они ничего не покупали в течение 31 дней. Однако, в реальности это могли быть уже активные клиенты. Просто мы не видим их ранние данные. В рамках текущей задачи эта возможная ошибка классификации не учитывается. Ее можно было бы избежать отсечением первых 31 дней в тестовых данных.
* Если до первого дня активности клиент не был активен более 31 дня, то этот период считается отточным для клиента. Однако, в реальности это мог быть клиент, впервые сделавший покупку (новый клиент). В рамках текущей задачи эта возможная ошибка классификации не учитывается из-за недостатка информации по данным.

### 2.3. Методика прогнозирования вероятности оттока клиентов

В качестве стратегии прогнозирования отточных клиентов выбирается построение таблицы еженедневной активности клиентов. Для каждого дня агрегируются данные по общим предпочтениям клиента (портрет клиента) и его активности за предыдущие периоды до 31 дней. Прогноз вероятности оттока делается для текущего дня.

Модель прогнозирования будет построена при помощи машинного обучения. Будут обучены и протестированы несколько популярных моделей классификации из пакета scikit-learn с функционалом predict_proba (предсказание вероятности) и на базовых гиперпараметрах:
* AdaBoostClassifier
* BaggingClassifier
* DecisionTreeClassifier
* GradientBoostingClassifier
* KNeighborsClassifier
* LogisticRegression
* RandomForestClassifier

Далее несколько самых точных моделей будут обучены и протестированы с оптимальными гиперпараметрами.

Поскольку задача состоит в том, чтобы корректно идентифицировать отточных клиентов, требуется максимизировать получение True Positive результатов классификации. Мы можем исключить False Positive ошибки (клиенты, которые, были классифицированы отточными, но этого не произошло), поскольку они не так важны.
Кроме того, требуется минимизировать False Negative ошибку (клиенты, которые, были классифицированы как активные, но они ушли). В этом случае мы можем потерять этих клиентов из-за ошибки.

Соответственно, основные задачи построения моделей: максимизация True Positive и минимизация False Negative.

Метрики оптимизации моделей:
* Кривая ROC-AUC
* Confusion Matrix

# 3. Формирование целевой переменной

### 3.1. Базовая таблица ежедневной активности клиентов

Создадим базовую таблицу ежедневной активности клиентов.

Для определения отточных клиентов 1 группы потребуется признак активности клиента 'is_active' (1 - есть покупки в текущий день, 0 - нет покупок). Для определения отточных клиентов 2 группы потребуется признак высокой активности клиента 'is_high_active' (1 - количество покупок выше порога в текущий день, 0 - ниже или равны порогу).

In [14]:
days = pd.date_range(full.chq_date.min(), full.chq_date.max(), freq='D')
ROLL_MONTH = 31
df_list = []
tmp = full[['chq_date', 'client_id', 'chq_id', 'sales_sum']]
all_clients = tmp.client_id.unique()

with tqdm (total=full.client_id.nunique()) as pbar:
    for client in all_clients:
        result = pd.DataFrame({'chq_date':days, 'client_id':client})
        tmp2 = tmp[tmp.client_id==client]
        
        # Создание целевой переменной для 1 группы отточных клиентов
        tmp3 = tmp2.groupby(['chq_date'])['chq_id'].nunique().rename('chq_count_today').reset_index()
        result = result.merge(tmp3, how='left', on=['chq_date']).fillna(0)
        result['is_active'] = np.where(result['chq_count_today']>0, 1, 0)
        
        # Итерируем по всем строкам и рассчитываем целевую переменную по 1 группе
        result['is_churn_grp1'] = np.NaN
        inactive_streak = 0
        index_list = []
        for index, row in result.iterrows():
            index_list.append(index)
            if ((row['is_active'] == 1) & (inactive_streak < ROLL_MONTH)):
                for i in index_list:
                    result.at[i, 'is_churn_grp1'] = 0
                inactive_streak = 0
                index_list = []
            elif ((row['is_active'] == 1) & (inactive_streak >= ROLL_MONTH)):
                for i in index_list:
                    result.at[i, 'is_churn_grp1'] = 1
                result.at[index, 'is_churn_grp1'] = 0
                inactive_streak = 0
                index_list = []
            elif ((row['is_active'] == 0) & (inactive_streak < ROLL_MONTH)):
                inactive_streak = inactive_streak + 1
            elif ((row['is_active'] == 0) & (inactive_streak >= ROLL_MONTH)):
                inactive_streak = inactive_streak + 1
                for i in index_list:
                    result.at[i, 'is_churn_grp1'] = 1
                index_list = []        

        # Создание целевой переменной для 2 группы отточных клиентов
        threshold = tmp2.groupby([pd.Grouper(key='chq_date', freq='1D')])['sales_sum']\
                .sum()\
                .rename('sum_today')\
                .reset_index()['sum_today']\
                .replace(0, np.NaN)\
                .mean()*0.25
        tmp3 = tmp2.groupby('chq_date')['sales_sum']\
                .sum()\
                .rename('sum_today')\
                .reset_index()
        result = result.merge(tmp3, how='left', on=['chq_date']).fillna(0)
        result['is_high_active'] = result['sum_today'].apply(lambda x: 1 if x>threshold else 0)
        
        # Итерируем по всем строкам и рассчитываем целевую переменную по 2 группе
        result['is_churn_grp2'] = np.NaN
        inactive_streak = 0
        index_list = []
        for index, row in result.iterrows():
            index_list.append(index)
            if ((row['is_high_active'] == 1) & (inactive_streak < ROLL_MONTH)):
                for i in index_list:
                    result.at[i, 'is_churn_grp2'] = 0
                inactive_streak = 0
                index_list = []
            elif ((row['is_high_active'] == 1) & (inactive_streak >= ROLL_MONTH)):
                for i in index_list:
                    result.at[i, 'is_churn_grp2'] = 1
                result.at[index, 'is_churn_grp2'] = 0
                inactive_streak = 0
                index_list = []
            elif ((row['is_high_active'] == 0) & (inactive_streak < ROLL_MONTH)):
                inactive_streak = inactive_streak + 1
            elif ((row['is_high_active'] == 0) & (inactive_streak >= ROLL_MONTH)):
                inactive_streak = inactive_streak + 1
                for i in index_list:
                    result.at[i, 'is_churn_grp2'] = 1
                index_list = []        

        df_list.append(result)
        pbar.update()
data = pd.concat(df_list, axis=0, ignore_index=True)
data['is_churn'] = data[['is_churn_grp1', 'is_churn_grp2']].max(axis=1)
data = data.drop(columns=['is_churn_grp1', 'is_churn_grp2'])

### Результат

In [42]:
data.head()

,chq_date,client_id,chq_count_today,is_active,sum_today,is_high_active,is_churn
0,2016-10-04,56437,0.0,0,0.0,0,0.0
1,2016-10-05,56437,0.0,0,0.0,0,0.0
2,2016-10-06,56437,0.0,0,0.0,0,0.0
3,2016-10-07,56437,0.0,0,0.0,0,0.0
4,2016-10-08,56437,0.0,0,0.0,0,0.0


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36598170 entries, 0 to 36598169
Data columns (total 7 columns):
chq_date           datetime64[ns]
client_id          int64
chq_count_today    float64
is_active          int32
sum_today          float32
is_high_active     int64
is_churn           float64
dtypes: datetime64[ns](1), float32(1), float64(2), int32(1), int64(2)
memory usage: 1.6 GB


# 4. Сохранение данных

Сохраним финальные данные в формате pickle для последующего использования.

In [15]:
path = str(workdir+'/data')
if not os.path.isdir(path):
    os.mkdir(path)
    print('Папка успешно создана!')
else:
    print('Папка уже существует')

Папка уже существует


In [16]:
full.to_pickle(path+'/full.pkl')
data.to_pickle(path+'/target.pkl')